In [ ]:
"""
DrugReflector 基座模型训练
严格遵循 Science 2025 Supplementary Materials (Pages 2-4)

模型架构：
- Input: 978 (landmark genes)
- Hidden 1: 1,024 nodes + ReLU + Dropout(0.64) + BatchNorm
- Hidden 2: 2,048 nodes + ReLU + Dropout(0.64) + BatchNorm
- Output: 9,597 (compounds)

训练配置：
- Focal Loss (γ=2)
- Cosine Annealing with Warm Restarts
- 3-Fold Ensemble
- 50 epochs per model
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
from typing import Dict, Tuple, List
from sklearn.metrics import accuracy_score, top_k_accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


# ============================================================================
# 数据预处理
# ============================================================================

def clip_and_normalize_signature(X: np.ndarray, clip_range=(-2, 2)) -> np.ndarray:
    """
    裁剪并标准化签名，使其标准差为1
    
    原文（SI第3页）：
    "every transcriptional vector v is clipped to range [-2,2] such that 
    its standard deviation after clipping equals 1"
    
    Parameters:
    -----------
    X: np.ndarray
        原始表达矩阵 (n_samples, n_genes)
    clip_range: tuple
        裁剪范围，默认[-2, 2]
    
    Returns:
    --------
    np.ndarray: 处理后的矩阵
    """
    print(f"\n📊 Clipping and normalizing signatures...")
    print(f"   Clip range: {clip_range}")
    
    X_processed = np.zeros_like(X)
    
    for i in range(X.shape[0]):
        # 裁剪到指定范围
        vec = np.clip(X[i], clip_range[0], clip_range[1])
        
        # 标准化使标准差为1
        std = np.std(vec)
        if std > 0:
            vec = vec / std
        
        X_processed[i] = vec
    
    print(f"   ✓ Mean std after normalization: {np.std(X_processed, axis=1).mean():.4f}")
    print(f"   ✓ Data range after clipping: [{X_processed.min():.2f}, {X_processed.max():.2f}]")
    
    return X_processed


# ============================================================================
# PyTorch Dataset
# ============================================================================

class LINCSDataset(Dataset):
    """LINCS数据集类"""
    
    def __init__(self, X: np.ndarray, y: np.ndarray, fold_mask: np.ndarray = None):
        """
        Parameters:
        -----------
        X: np.ndarray
            表达矩阵 (n_samples, 978)
        y: np.ndarray
            化合物标签 (n_samples,)
        fold_mask: np.ndarray
            Fold掩码，True表示包含在此数据集中
        """
        if fold_mask is not None:
            self.X = torch.FloatTensor(X[fold_mask])
            self.y = torch.LongTensor(y[fold_mask])
        else:
            self.X = torch.FloatTensor(X)
            self.y = torch.LongTensor(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# ============================================================================
# Focal Loss
# ============================================================================

class FocalLoss(nn.Module):
    """
    Focal Loss实现
    
    原文（SI第3页）：
    "The models were trained using a focal loss function with a 
    focusing parameter of γ = 2"
    
    Reference: Lin et al. "Focal Loss for Dense Object Detection"
    """
    
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        """
        Parameters:
        -----------
        gamma: float
            聚焦参数，原文使用2.0
        alpha: Tensor or None
            类别权重
        reduction: str
            'mean', 'sum', or 'none'
        """
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        """
        Parameters:
        -----------
        inputs: Tensor
            模型输出 logits (N, C)
        targets: Tensor
            真实标签 (N,)
        """
        # 计算交叉熵
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        
        # 计算pt
        pt = torch.exp(-ce_loss)
        
        # Focal loss
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        
        if self.alpha is not None:
            alpha_t = self.alpha[targets]
            focal_loss = alpha_t * focal_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


# ============================================================================
# 模型架构
# ============================================================================

class DrugReflectorModel(nn.Module):
    """
    DrugReflector神经网络架构
    
    原文（SI第2页）：
    "The input layer has 978 nodes (one for each landmark gene), 
    and the output layer has 9,597 nodes (one for each target LINCS perturbation). 
    The first hidden layer has 1,024 nodes, and the second has 2,048 nodes 
    using rectified linear units (ReLU) to compute node activations."
    """
    
    def __init__(
        self, 
        input_size=978, 
        hidden1_size=1024, 
        hidden2_size=2048, 
        output_size=9597,
        dropout_rate=0.64,
        batch_norm_momentum=0.1
    ):
        """
        Parameters:
        -----------
        input_size: int
            输入特征数（landmark genes）
        hidden1_size: int
            第一隐藏层大小
        hidden2_size: int
            第二隐藏层大小
        output_size: int
            输出类别数（化合物数）
        dropout_rate: float
            Dropout比率，原文0.64
        batch_norm_momentum: float
            BatchNorm动量，原文0.1
        """
        super(DrugReflectorModel, self).__init__()
        
        # 第一隐藏层
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.bn1 = nn.BatchNorm1d(hidden1_size, momentum=batch_norm_momentum)
        self.dropout1 = nn.Dropout(dropout_rate)
        
        # 第二隐藏层
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.bn2 = nn.BatchNorm1d(hidden2_size, momentum=batch_norm_momentum)
        self.dropout2 = nn.Dropout(dropout_rate)
        
        # 输出层
        self.fc3 = nn.Linear(hidden2_size, output_size)
        
        # 初始化权重
        self._initialize_weights()
    
    def _initialize_weights(self):
        """权重初始化"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        """
        前向传播
        
        Parameters:
        -----------
        x: Tensor
            输入 (batch_size, 978)
        
        Returns:
        --------
        Tensor: 输出logits (batch_size, 9597)
        """
        # 第一隐藏层
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        
        # 第二隐藏层
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        
        # 输出层
        x = self.fc3(x)
        
        return x


# ============================================================================
# 训练器
# ============================================================================

class DrugReflectorTrainer:
    """
    DrugReflector训练器
    
    实现3-fold ensemble训练策略
    """
    
    def __init__(
        self,
        device='cuda' if torch.cuda.is_available() else 'cpu',
        initial_lr=0.0139,
        min_lr=0.00001,
        weight_decay=1e-5,
        t_0=20,  # Time to first restart
        t_mult=1.9,  # 原文0.5，但这样会导致周期减半，实际应该>1
        focal_gamma=2.0,
        batch_size=256,
        num_epochs=50,
        early_stop_epoch=20
    ):
        """
        Parameters:
        -----------
        device: str
            计算设备
        initial_lr: float
            初始学习率（原文：0.0139）
        min_lr: float
            最小学习率（原文：0.00001）
        weight_decay: float
            权重衰减（原文：1e-5）
        t_0: int
            第一次warm restart前的epoch数（原文：20）
        t_mult: float
            warm restart周期倍增因子（原文：0.5）
        focal_gamma: float
            Focal loss聚焦参数（原文：2.0）
        batch_size: int
            批次大小
        num_epochs: int
            总训练轮数（原文：50）
        early_stop_epoch: int
            早停检查点（原文：20）
        """
        self.device = device
        self.initial_lr = initial_lr
        self.min_lr = min_lr
        self.weight_decay = weight_decay
        self.t_0 = t_0
        self.t_mult = t_mult
        self.focal_gamma = focal_gamma
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.early_stop_epoch = early_stop_epoch
        
        print(f"\n🚀 DrugReflector Trainer initialized")
        print(f"   Device: {self.device}")
        print(f"   Initial LR: {self.initial_lr}")
        print(f"   Min LR: {self.min_lr}")
        print(f"   Weight Decay: {self.weight_decay}")
        print(f"   T_0 (first restart): {self.t_0}")
        print(f"   Focal γ: {self.focal_gamma}")
        print(f"   Batch size: {self.batch_size}")
        print(f"   Epochs: {self.num_epochs}")
    
    def train_single_model(
        self,
        model: nn.Module,
        train_loader: DataLoader,
        val_loader: DataLoader,
        fold_id: int
    ) -> Dict:
        """
        训练单个模型
        
        Parameters:
        -----------
        model: nn.Module
            DrugReflector模型
        train_loader: DataLoader
            训练数据加载器
        val_loader: DataLoader
            验证数据加载器
        fold_id: int
            Fold ID (0, 1, 2)
        
        Returns:
        --------
        Dict: 训练历史
        """
        print(f"\n{'='*80}")
        print(f"Training Model {fold_id}")
        print(f"{'='*80}")
        
        # 损失函数和优化器
        criterion = FocalLoss(gamma=self.focal_gamma)
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=self.initial_lr,
            weight_decay=self.weight_decay
        )
        
        # 学习率调度器
        scheduler = CosineAnnealingWarmRestarts(
            optimizer,
            T_0=self.t_0,
            T_mult=int(self.t_mult) if self.t_mult >= 1 else 1,
            eta_min=self.min_lr
        )
        
        # 训练历史
        history = {
            'train_loss': [],
            'val_loss': [],
            'val_recall': [],
            'val_top1_acc': [],
            'val_top10_acc': [],
            'learning_rates': []
        }
        
        best_recall = 0.0
        best_epoch = 0
        best_model_state = None
        
        # 训练循环
        for epoch in range(self.num_epochs):
            # ========== 训练阶段 ==========
            model.train()
            train_loss = 0.0
            
            pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{self.num_epochs}")
            for batch_X, batch_y in pbar:
                batch_X = batch_X.to(self.device)
                batch_y = batch_y.to(self.device)
                
                # 前向传播
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                
                # 反向传播
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
                pbar.set_postfix({'loss': loss.item()})
            
            avg_train_loss = train_loss / len(train_loader)
            
            # ========== 验证阶段 ==========
            model.eval()
            val_loss = 0.0
            all_preds = []
            all_labels = []
            all_probs = []
            
            with torch.no_grad():
                for batch_X, batch_y in val_loader:
                    batch_X = batch_X.to(self.device)
                    batch_y = batch_y.to(self.device)
                    
                    outputs = model(batch_X)
                    loss = criterion(outputs, batch_y)
                    val_loss += loss.item()
                    
                    # 计算预测
                    probs = F.softmax(outputs, dim=1)
                    preds = torch.argmax(probs, dim=1)
                    
                    all_preds.append(preds.cpu().numpy())
                    all_labels.append(batch_y.cpu().numpy())
                    all_probs.append(probs.cpu().numpy())
            
            avg_val_loss = val_loss / len(val_loader)
            
            # 计算指标
            all_preds = np.concatenate(all_preds)
            all_labels = np.concatenate(all_labels)
            all_probs = np.concatenate(all_probs)
            
            top1_acc = accuracy_score(all_labels, all_preds)
            top10_acc = top_k_accuracy_score(all_labels, all_probs, k=10)
            
            # 计算Top 1% recall (原文主要指标)
            top1_percent_k = max(1, int(0.01 * all_probs.shape[1]))
            recall = top_k_accuracy_score(all_labels, all_probs, k=top1_percent_k)
            
            # 记录历史
            current_lr = optimizer.param_groups[0]['lr']
            history['train_loss'].append(avg_train_loss)
            history['val_loss'].append(avg_val_loss)
            history['val_recall'].append(recall)
            history['val_top1_acc'].append(top1_acc)
            history['val_top10_acc'].append(top10_acc)
            history['learning_rates'].append(current_lr)
            
            # 打印进度
            print(f"\nEpoch {epoch+1}/{self.num_epochs}")
            print(f"  Train Loss: {avg_train_loss:.4f}")
            print(f"  Val Loss: {avg_val_loss:.4f}")
            print(f"  Val Recall (top 1%): {recall:.4f}")
            print(f"  Val Top-1 Acc: {top1_acc:.4f}")
            print(f"  Val Top-10 Acc: {top10_acc:.4f}")
            print(f"  Learning Rate: {current_lr:.6f}")
            
            # 保存最佳模型
            if recall > best_recall:
                best_recall = recall
                best_epoch = epoch
                best_model_state = model.state_dict().copy()
                print(f"  ✓ New best model! (Recall: {best_recall:.4f})")
            
            # 学习率调度
            scheduler.step()
        
        # 加载最佳模型
        if best_model_state is not None:
            model.load_state_dict(best_model_state)
            print(f"\n✓ Loaded best model from epoch {best_epoch+1} "
                  f"(Recall: {best_recall:.4f})")
        
        return history
    
    def train_ensemble(
        self,
        training_data: Dict,
        output_dir: Path
    ) -> List[nn.Module]:
        """
        训练3-fold ensemble
        
        原文（SI第2-3页）：
        "The training data was divided randomly into three folds, with perturbation 
        replicates balanced across the folds. Models were independently trained on 
        two of three folds."
        
        Parameters:
        -----------
        training_data: Dict
            包含X, y, folds的训练数据
        output_dir: Path
            模型保存目录
        
        Returns:
        --------
        List[nn.Module]: 训练好的3个模型
        """
        X = training_data['X']
        y = training_data['y']
        folds = training_data['folds']
        n_compounds = len(training_data['compound_names'])
        
        # 预处理数据
        X_processed = clip_and_normalize_signature(X)
        
        print(f"\n{'='*80}")
        print(f"🎯 Training 3-Fold Ensemble")
        print(f"{'='*80}")
        print(f"  Total samples: {len(X):,}")
        print(f"  Total compounds: {n_compounds:,}")
        print(f"  Input features: {X.shape[1]}")
        
        models = []
        histories = []
        
        # 训练3个模型
        for fold_id in range(3):
            print(f"\n{'='*80}")
            print(f"Training Fold {fold_id} Model")
            print(f"{'='*80}")
            
            # 准备数据
            # 模型在fold_id上验证，在其他两个fold上训练
            val_mask = folds == fold_id
            train_mask = ~val_mask
            
            print(f"  Training samples: {train_mask.sum():,}")
            print(f"  Validation samples: {val_mask.sum():,}")
            
            # 创建数据集
            train_dataset = LINCSDataset(X_processed, y, train_mask)
            val_dataset = LINCSDataset(X_processed, y, val_mask)
            
            train_loader = DataLoader(
                train_dataset,
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=4,
                pin_memory=True
            )
            
            val_loader = DataLoader(
                val_dataset,
                batch_size=self.batch_size,
                shuffle=False,
                num_workers=4,
                pin_memory=True
            )
            
            # 创建模型
            model = DrugReflectorModel(
                input_size=X.shape[1],
                output_size=n_compounds,
                dropout_rate=0.64,
                batch_norm_momentum=0.1
            ).to(self.device)
            
            print(f"\n  Model architecture:")
            print(f"    Input: {X.shape[1]} → Hidden1: 1024 → Hidden2: 2048 → Output: {n_compounds}")
            print(f"    Parameters: {sum(p.numel() for p in model.parameters()):,}")
            
            # 训练模型
            history = self.train_single_model(
                model,
                train_loader,
                val_loader,
                fold_id
            )
            
            models.append(model)
            histories.append(history)
            
            # 保存模型
            model_path = output_dir / f"model_fold{fold_id}.pt"
            torch.save({
                'model_state_dict': model.state_dict(),
                'fold_id': fold_id,
                'history': history,
                'config': {
                    'input_size': X.shape[1],
                    'output_size': n_compounds,
                    'dropout_rate': 0.64,
                    'batch_norm_momentum': 0.1
                }
            }, model_path)
            print(f"\n  ✓ Model saved to {model_path}")
        
        # 保存ensemble历史
        ensemble_history_path = output_dir / "ensemble_history.pkl"
        with open(ensemble_history_path, 'wb') as f:
            pickle.dump(histories, f)
        
        print(f"\n{'='*80}")
        print(f"✅ Ensemble Training Complete!")
        print(f"{'='*80}")
        print(f"  Models saved to: {output_dir}")
        
        return models, histories


# ============================================================================
# 模型评估
# ============================================================================

class DrugReflectorEvaluator:
    """模型评估器"""
    
    def __init__(self, models: List[nn.Module], device='cuda'):
        self.models = models
        self.device = device
        for model in self.models:
            model.eval()
    
    def predict_ensemble(self, X: np.ndarray) -> np.ndarray:
        """
        Ensemble预测
        
        原文（SI第2页）：
        "The final predicted class probabilities were the softmax probabilities 
        of the average score over all three folds."
        
        Parameters:
        -----------
        X: np.ndarray
            输入数据 (n_samples, 978)
        
        Returns:
        --------
        np.ndarray: 平均得分 (n_samples, n_compounds)
        """
        X_tensor = torch.FloatTensor(X).to(self.device)
        
        all_scores = []
        with torch.no_grad():
            for model in self.models:
                scores = model(X_tensor)
                all_scores.append(scores.cpu().numpy())
        
        # 平均得分
        avg_scores = np.mean(all_scores, axis=0)
        
        return avg_scores
    
    def evaluate(
        self,
        X: np.ndarray,
        y: np.ndarray,
        compound_names: List[str]
    ) -> Dict:
        """
        评估ensemble性能
        
        Parameters:
        -----------
        X: np.ndarray
            测试数据
        y: np.ndarray
            真实标签
        compound_names: List[str]
            化合物名称列表
        
        Returns:
        --------
        Dict: 评估指标
        """
        print(f"\n📊 Evaluating ensemble performance...")
        
        # 预处理
        X_processed = clip_and_normalize_signature(X)
        
        # 预测
        avg_scores = self.predict_ensemble(X_processed)
        probs = torch.softmax(torch.FloatTensor(avg_scores), dim=1).numpy()
        preds = np.argmax(avg_scores, axis=1)
        
        # 计算指标
        top1_acc = accuracy_score(y, preds)
        top10_acc = top_k_accuracy_score(y, probs, k=10)
        
        # Top 1% recall
        top1_percent_k = max(1, int(0.01 * probs.shape[1]))
        recall = top_k_accuracy_score(y, probs, k=top1_percent_k)
        
        print(f"  Top-1 Accuracy: {top1_acc:.4f}")
        print(f"  Top-10 Accuracy: {top10_acc:.4f}")
        print(f"  Top 1% Recall: {recall:.4f}")
        
        # 计算每个化合物的recall
        compound_recalls = []
        for compound_idx in range(len(compound_names)):
            mask = y == compound_idx
            if mask.sum() == 0:
                continue
            
            compound_probs = probs[mask]
            compound_labels = y[mask]
            
            if len(compound_probs) > 0:
                compound_recall = top_k_accuracy_score(
                    compound_labels, 
                    compound_probs, 
                    k=top1_percent_k
                )
                compound_recalls.append(compound_recall)
        
        avg_compound_recall = np.mean(compound_recalls)
        
        print(f"  Average per-compound recall: {avg_compound_recall:.4f}")
        
        results = {
            'top1_accuracy': top1_acc,
            'top10_accuracy': top10_acc,
            'top1_percent_recall': recall,
            'avg_compound_recall': avg_compound_recall,
            'compound_recalls': compound_recalls,
            'predictions': preds,
            'probabilities': probs
        }
        
        return results


# ============================================================================
# 可视化
# ============================================================================

def plot_training_history(histories: List[Dict], output_dir: Path):
    """
    绘制训练历史
    
    Parameters:
    -----------
    histories: List[Dict]
        3个模型的训练历史
    output_dir: Path
        输出目录
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    metrics = [
        ('train_loss', 'Training Loss', 'Loss'),
        ('val_loss', 'Validation Loss', 'Loss'),
        ('val_recall', 'Validation Recall (Top 1%)', 'Recall'),
        ('val_top1_acc', 'Top-1 Accuracy', 'Accuracy'),
        ('val_top10_acc', 'Top-10 Accuracy', 'Accuracy'),
        ('learning_rates', 'Learning Rate', 'LR')
    ]
    
    for idx, (metric, title, ylabel) in enumerate(metrics):
        ax = axes[idx // 3, idx % 3]
        
        for fold_id, history in enumerate(histories):
            epochs = range(1, len(history[metric]) + 1)
            ax.plot(epochs, history[metric], label=f'Fold {fold_id}', linewidth=2)
        
        ax.set_xlabel('Epoch')
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    save_path = output_dir / 'training_history.png'
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"\n✓ Training history plot saved to {save_path}")
    plt.close()


# ============================================================================
# 主训练流程
# ============================================================================

def main():
    """主训练流程"""
    
    print("=" * 80)
    print("🚀 DrugReflector Base Model Training")
    print("   Following Science 2025 Supplementary Materials")
    print("=" * 80)
    
    # 设置路径
    data_dir = Path("E:/科研/Models/drugreflector/processed_data")
    output_dir = Path("E:/科研/Models/drugreflector/trained_models")
    output_dir.mkdir(exist_ok=True, parents=True)
    
    # 加载数据
    print(f"\n📂 Loading preprocessed data...")
    data_file = data_dir / "training_data_paper_compliant.pkl"
    
    if not data_file.exists():
        print(f"❌ Data file not found: {data_file}")
        print(f"   Please run the preprocessing script first.")
        return
    
    with open(data_file, 'rb') as f:
        training_data = pickle.load(f)
    
    print(f"✓ Loaded training data")
    print(f"  Samples: {len(training_data['X']):,}")
    print(f"  Compounds: {len(training_data['compound_names']):,}")
    print(f"  Features: {training_data['X'].shape[1]}")
    
    # 创建训练器
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"\n🔧 Initializing trainer...")
    print(f"  Device: {device}")
    
    if device == 'cuda':
        print(f"  GPU: {torch.cuda.get_device_name(0)}")
        print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    trainer = DrugReflectorTrainer(
        device=device,
        initial_lr=0.0139,
        min_lr=0.00001,
        weight_decay=1e-5,
        t_0=20,
        focal_gamma=2.0,
        batch_size=256,
        num_epochs=50,
        early_stop_epoch=20
    )
    
    # 训练ensemble
    models, histories = trainer.train_ensemble(training_data, output_dir)
    
    # 可视化训练历史
    print(f"\n📊 Plotting training history...")
    plot_training_history(histories, output_dir)
    
    # 评估模型
    print(f"\n📊 Evaluating ensemble on validation sets...")
    evaluator = DrugReflectorEvaluator(models, device)
    
    # 在每个fold上评估
    for fold_id in range(3):
        val_mask = training_data['folds'] == fold_id
        X_val = training_data['X'][val_mask]
        y_val = training_data['y'][val_mask]
        
        print(f"\n{'='*80}")
        print(f"Fold {fold_id} Validation Performance")
        print(f"{'='*80}")
        
        results = evaluator.evaluate(
            X_val,
            y_val,
            training_data['compound_names']
        )
        
        # 保存结果
        results_path = output_dir / f"fold{fold_id}_results.pkl"
        with open(results_path, 'wb') as f:
            pickle.dump(results, f)
        print(f"  ✓ Results saved to {results_path}")
    
    print(f"\n{'='*80}")
    print(f"✅ TRAINING COMPLETE!")
    print(f"{'='*80}")
    print(f"📁 All outputs saved to: {output_dir}")
    print(f"\nFiles created:")
    print(f"  • model_fold0.pt - Fold 0 model checkpoint")
    print(f"  • model_fold1.pt - Fold 1 model checkpoint")
    print(f"  • model_fold2.pt - Fold 2 model checkpoint")
    print(f"  • ensemble_history.pkl - Training history")
    print(f"  • training_history.png - Training curves")
    print(f"  • fold*_results.pkl - Evaluation results")


if __name__ == "__main__":
    main()

# DrugReflector 基座模型训练详细说明

## 📋 目录
1. [训练流程概述](#训练流程概述)
2. [关键技术细节](#关键技术细节)
3. [代码结构](#代码结构)
4. [使用方法](#使用方法)
5. [超参数说明](#超参数说明)
6. [常见问题](#常见问题)
7. [性能优化](#性能优化)

---

## 训练流程概述

DrugReflector采用**3-fold ensemble**策略训练基座模型，完全按照Science 2025补充材料第2-4页的方法实现。

### 整体流程

```
输入数据 (425,242 × 978)
    ↓
数据标准化（clip & normalize）
    ↓
3-fold 划分
    ↓
┌─────────────┬─────────────┬─────────────┐
│  Model 0    │  Model 1    │  Model 2    │
│ Train: 1,2  │ Train: 0,2  │ Train: 0,1  │
│ Val:   0    │ Val:   1    │ Val:   2    │
└─────────────┴─────────────┴─────────────┘
    ↓
Ensemble预测（平均得分）
    ↓
最终输出
```

---

## 关键技术细节

### 1. 数据预处理 ⭐

**原文（SI 第3页）：**
> "every transcriptional vector v is clipped to range [-2,2] such that its standard deviation after clipping equals 1"

**实现：**
```python
def clip_and_normalize_signature(X, clip_range=(-2, 2)):
    X_processed = np.zeros_like(X)
    for i in range(X.shape[0]):
        # Step 1: 裁剪到[-2, 2]
        vec = np.clip(X[i], clip_range[0], clip_range[1])
        
        # Step 2: 标准化使标准差为1
        std = np.std(vec)
        if std > 0:
            vec = vec / std
        
        X_processed[i] = vec
    return X_processed
```

**为什么这样做？**
- **裁剪**：去除极端异常值，避免模型被outlier主导
- **标准化**：确保所有样本在相同尺度，提高训练稳定性
- **混合表示**：介于二值化（上调/下调）和连续值之间

---

### 2. 模型架构 🏗️

**原文（SI 第2页）：**
> "The input layer has 978 nodes (one for each landmark gene), and the output layer has 9,597 nodes (one for each target LINCS perturbation). The first hidden layer has 1,024 nodes, and the second has 2,048 nodes using rectified linear units (ReLU)."

**完整架构：**

```
Input (978 genes)
    ↓
Linear(978 → 1024)
    ↓
BatchNorm1d(momentum=0.1)
    ↓
ReLU
    ↓
Dropout(p=0.64)
    ↓
Linear(1024 → 2048)
    ↓
BatchNorm1d(momentum=0.1)
    ↓
ReLU
    ↓
Dropout(p=0.64)
    ↓
Linear(2048 → 9597)
    ↓
Output (9597 compounds)
```

**参数量：**
- Layer 1: 978 × 1024 = 1,001,472
- Layer 2: 1024 × 2048 = 2,097,152
- Layer 3: 2048 × 9597 = 19,654,656
- **总计：~22.7M 参数**

---

### 3. Focal Loss 🎯

**原文（SI 第3页）：**
> "The models were trained using a focal loss function with a focusing parameter of γ = 2"

**公式：**
```
FL(pt) = -(1 - pt)^γ * log(pt)

其中：
- pt = softmax(logits)[true_class]
- γ = 2.0 (focusing parameter)
```

**为什么使用Focal Loss？**
1. **类别不平衡**：9,597个化合物，样本分布极不均匀
2. **关注难样本**：(1-pt)^γ 权重使模型更关注错分样本
3. **减少简单样本贡献**：易分样本的loss被大幅降低

**与交叉熵对比：**
```
样本置信度    Cross Entropy Loss    Focal Loss (γ=2)
  0.99              0.01                 0.0001  (↓100x)
  0.90              0.11                 0.0011  (↓100x)
  0.50              0.69                 0.1725  (↓4x)
  0.10              2.30                 1.863   (相近)
```

---

### 4. 学习率调度 📈

**原文（SI 第3-4页）：**
> "The learning rate was determined by a cosine annealing schedule with warm restarts, with 20 epochs before the first restart, an initial learning rate of 0.0139, and a minimum learning rate of 0.00001."

**Cosine Annealing with Warm Restarts：**

```python
scheduler = CosineAnnealingWarmRestarts(
    optimizer,
    T_0=20,           # 第一次重启前的epoch数
    T_mult=1,         # 周期倍增因子
    eta_min=0.00001   # 最小学习率
)
```

**学习率变化曲线：**
```
LR
 |
0.0139 |    ╱╲         ╱╲         ╱╲
       |   ╱  ╲       ╱  ╲       ╱  ╲
       |  ╱    ╲     ╱    ╲     ╱    ╲
0.00001|─╱──────╲───╱──────╲───╱──────╲──
       |          ╲ ╱        ╲ ╱        ╲
       └────────────┴─────────┴──────────┴── Epoch
       0          20         40         50
       
       ← T_0=20 →  ← T_0=20 →  ← T_0=20 →
```

**Warm Restart的好处：**
1. **逃离局部最优**：周期性提高LR帮助跳出局部最小值
2. **多次精细搜索**：每次降低时都能细化参数
3. **ensemble效应**：不同restart阶段的模型可以互补

---

### 5. 3-Fold Ensemble 🎭

**原文（SI 第2页）：**
> "The training data was divided randomly into three folds, with perturbation replicates balanced across the folds. Models were independently trained on two of three folds."

**划分策略：**
```python
# 关键：同一化合物的所有replicate在同一fold
for pert_id in unique_compounds:
    pert_indices = np.where(sample_meta['pert_id'] == pert_id)[0]
    # 随机打乱并均匀分配到3个fold
    randomly_assign_to_folds(pert_indices)
```

**训练和验证：**
| Model | Training Folds | Validation Fold | Samples (Train) | Samples (Val) |
|-------|----------------|-----------------|-----------------|---------------|
| 0     | 1, 2           | 0               | ~283k           | ~142k         |
| 1     | 0, 2           | 1               | ~283k           | ~142k         |
| 2     | 0, 1           | 2               | ~283k           | ~142k         |

**Ensemble预测：**
```python
# 平均3个模型的logits（注意：不是概率）
avg_scores = (model0_logits + model1_logits + model2_logits) / 3

# 然后计算softmax
final_probs = softmax(avg_scores)

# 排序得到最终预测
ranks = argsort(-final_probs)
```

---

## 代码结构

```
drugreflector_training.py
├── clip_and_normalize_signature()    # 数据预处理
├── LINCSDataset                       # PyTorch数据集
├── FocalLoss                          # Focal Loss实现
├── DrugReflectorModel                 # 模型架构
├── DrugReflectorTrainer               # 训练器
│   ├── train_single_model()          # 单模型训练
│   └── train_ensemble()              # 3-fold ensemble训练
├── DrugReflectorEvaluator            # 评估器
│   ├── predict_ensemble()            # Ensemble预测
│   └── evaluate()                    # 性能评估
├── plot_training_history()           # 可视化
└── main()                            # 主流程
```

---

## 使用方法

### 基础使用

```bash
# 1. 确保已经完成数据预处理
# 运行 drugreflector_preprocessing_optimized.py

# 2. 运行训练
python drugreflector_training.py
```

### 自定义训练

```python
from drugreflector_training import DrugReflectorTrainer
import pickle
from pathlib import Path

# 加载数据
with open('training_data_paper_compliant.pkl', 'rb') as f:
    training_data = pickle.load(f)

# 创建训练器（自定义参数）
trainer = DrugReflectorTrainer(
    device='cuda',
    initial_lr=0.02,        # 更高的学习率
    min_lr=0.0001,
    weight_decay=1e-4,      # 更强的正则化
    t_0=15,                 # 更早的restart
    focal_gamma=2.5,        # 更强的focusing
    batch_size=512,         # 更大的batch
    num_epochs=100          # 更多epoch
)

# 训练
output_dir = Path("./my_models")
output_dir.mkdir(exist_ok=True)
models, histories = trainer.train_ensemble(training_data, output_dir)
```

### 加载已训练模型

```python
import torch
from drugreflector_training import DrugReflectorModel

# 加载单个模型
checkpoint = torch.load('model_fold0.pt')
model = DrugReflectorModel(
    input_size=978,
    output_size=9597
)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# 加载ensemble
models = []
for fold_id in range(3):
    checkpoint = torch.load(f'model_fold{fold_id}.pt')
    model = DrugReflectorModel(input_size=978, output_size=9597)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    models.append(model)
```

### 使用模型进行预测

```python
from drugreflector_training import DrugReflectorEvaluator
import numpy as np

# 创建评估器
evaluator = DrugReflectorEvaluator(models, device='cuda')

# 准备查询signature（例如：MEP → MPC转换）
query_signature = np.random.randn(1, 978)  # 示例数据

# 预测
scores = evaluator.predict_ensemble(query_signature)
probs = torch.softmax(torch.FloatTensor(scores), dim=1).numpy()

# 获取top-k预测
top_k = 100
top_indices = np.argsort(-probs[0])[:top_k]
top_probs = probs[0, top_indices]

# 映射到化合物名称
top_compounds = [compound_names[idx] for idx in top_indices]

for i, (compound, prob) in enumerate(zip(top_compounds, top_probs)):
    print(f"{i+1}. {compound}: {prob:.4f}")
```

---

## 超参数说明

### 原文超参数（Table S5）

| 参数 | 搜索范围 | 最优值 | 说明 |
|------|----------|--------|------|
| **Dropout** | (0.2, 0.8] | 0.64 | 防止过拟合 |
| **Initial LR** | (1e-4, 1e-1] | 0.0139 | 初始学习率 |
| **Weight Decay** | (1e-7, 1e-1] | 1e-5 | L2正则化 |
| **T_0** | 10-50 | 20 | 第一次restart的epoch数 |

### 超参数调优建议

#### 1. **Dropout (0.64)**
```python
# 数据量大 → 可以降低dropout
dropout_rate = 0.5  # 如果数据>1M样本

# 数据量小 → 提高dropout
dropout_rate = 0.7  # 如果数据<100k样本
```

#### 2. **Learning Rate (0.0139)**
```python
# GPU内存足够 → 增大batch size，提高LR
batch_size = 512
initial_lr = 0.02

# GPU内存不足 → 减小batch size，降低LR
batch_size = 128
initial_lr = 0.007
```

#### 3. **Focal Gamma (2.0)**
```python
# 类别极不平衡 → 提高gamma
focal_gamma = 2.5

# 类别相对均衡 → 降低gamma
focal_gamma = 1.5

# 不需要focal loss → 设为0（变成交叉熵）
focal_gamma = 0.0
```

#### 4. **Batch Size (256)**
```python
# 根据GPU内存调整
GPU_MEMORY_GB = torch.cuda.get_device_properties(0).total_memory / 1e9

if GPU_MEMORY_GB > 24:
    batch_size = 512
elif GPU_MEMORY_GB > 16:
    batch_size = 256
elif GPU_MEMORY_GB > 8:
    batch_size = 128
else:
    batch_size = 64
```

---

## 常见问题

### Q1: 训练时间太长怎么办？

**优化策略：**

1. **使用混合精度训练**（速度提升~2x）
```python
from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()

# 训练循环中
with autocast():
    outputs = model(batch_X)
    loss = criterion(outputs, batch_y)

scaler.scale(loss).backward()
scaler.step(optimizer)
scaler.update()
```

2. **增大batch size**
```python
# 在GPU内存允许的情况下
batch_size = 512  # 从256增加到512
```

3. **减少epoch数**
```python
num_epochs = 30  # 从50减少到30
```

**预期训练时间：**
- **单个GPU (RTX 3090)**: ~6-8小时/模型
- **3个模型总计**: ~18-24小时
- **使用混合精度**: ~9-12小时

---

### Q2: GPU内存不足怎么办？

**解决方案：**

1. **减小batch size**
```python
batch_size = 128  # 或更小
```

2. **梯度累积**
```python
accumulation_steps = 4  # 累积4个batch再更新
optimizer.zero_grad()

for i, (batch_X, batch_y) in enumerate(train_loader):
    outputs = model(batch_X)
    loss = criterion(outputs, batch_y) / accumulation_steps
    loss.backward()
    
    if (i + 1) % accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
```

3. **使用CPU训练**（慢但可行）
```python
trainer = DrugReflectorTrainer(device='cpu')
```

---

### Q3: 如何判断模型是否收敛？

**检查指标：**

1. **Training Loss持续下降**
```python
# 正常：从~8.0 → ~3.0
# 异常：震荡或上升
```

2. **Validation Recall提升**
```python
# 正常：从~0.20 → ~0.45-0.50
# 异常：不提升或下降（过拟合）
```

3. **Learning Rate曲线**
```python
# 查看warm restart是否工作
# 应该看到周期性的上升和下降
```

4. **早停检查**
```python
# 如果validation recall在20 epoch后不再提升
# 可以提前停止训练
```

---

### Q4: 如何提高模型性能？

**策略清单：**

- [ ] **数据质量**：确保预处理正确（尤其是相似度过滤）
- [ ] **更多训练数据**：放宽过滤条件获得更多样本
- [ ] **数据增强**：添加高斯噪声、dropout增强
- [ ] **模型容量**：增大隐藏层（1024→2048, 2048→4096）
- [ ] **Ensemble增强**：训练5-fold或10-fold ensemble
- [ ] **学习率调优**：使用学习率查找器
- [ ] **正则化调整**：调整dropout和weight decay
- [ ] **损失函数**：尝试label smoothing

---

## 性能优化

### 1. 多GPU训练

```python
# DataParallel (简单但效率不高)
model = nn.DataParallel(model)

# DistributedDataParallel (推荐)
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

# 初始化进程组
dist.init_process_group(backend='nccl')
local_rank = torch.distributed.get_rank()
torch.cuda.set_device(local_rank)

# 创建DDP模型
model = DrugReflectorModel(...).to(local_rank)
model = DDP(model, device_ids=[local_rank])
```

### 2. 数据加载优化

```python
train_loader = DataLoader(
    train_dataset,
    batch_size=256,
    shuffle=True,
    num_workers=8,           # 增加worker数
    pin_memory=True,         # 加速CPU→GPU传输
    prefetch_factor=2,       # 预取因子
    persistent_workers=True  # 保持worker进程
)
```

### 3. 编译模型（PyTorch 2.0+）

```python
# 使用torch.compile加速
model = torch.compile(model, mode='reduce-overhead')
```

### 4. 混合精度训练

```python
from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()

for batch_X, batch_y in train_loader:
    with autocast():  # 自动混合精度
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
    
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
```

---

## 预期结果

### 训练完成后的指标

根据论文，训练完成后应该达到：

| 指标 | CMap Touchstone | sciPlex3 | Internal |
|------|-----------------|----------|----------|
| **Top 1% Recall** | 0.46-0.50 | 0.30-0.35 | 0.60-0.65 |
| **Top-10 Accuracy** | 0.35-0.40 | 0.20-0.25 | 0.45-0.50 |

### 训练日志示例

```
================================================================================
Training Model 0
================================================================================

Epoch 1/50
  Train Loss: 7.2456
  Val Loss: 6.8934
  Val Recall (top 1%): 0.2134
  Val Top-1 Acc: 0.0234
  Val Top-10 Acc: 0.1456
  Learning Rate: 0.013900
  ✓ New best model! (Recall: 0.2134)

Epoch 10/50
  Train Loss: 4.5678
  Val Loss: 4.9234
  Val Recall (top 1%): 0.3567
  Val Top-1 Acc: 0.0567
  Val Top-10 Acc: 0.2345
  Learning Rate: 0.007234

...

Epoch 50/50
  Train Loss: 3.1234
  Val Loss: 3.8456
  Val Recall (top 1%): 0.4789
  Val Top-1 Acc: 0.0934
  Val Top-10 Acc: 0.3678
  Learning Rate: 0.000056

✓ Loaded best model from epoch 47 (Recall: 0.4823)
```

---

## 文件输出

训练完成后会生成以下文件：

```
trained_models/
├── model_fold0.pt              # Fold 0模型检查点
│   ├── model_state_dict        # 模型权重
│   ├── fold_id                 # Fold ID
│   ├── history                 # 训练历史
│   └── config                  # 模型配置
├── model_fold1.pt              # Fold 1模型检查点
├── model_fold2.pt              # Fold 2模型检查点
├── ensemble_history.pkl        # 3个模型的训练历史
├── training_history.png        # 训练曲线图
├── fold0_results.pkl           # Fold 0评估结果
├── fold1_results.pkl           # Fold 1评估结果
└── fold2_results.pkl           # Fold 2评估结果
```

---

## 引用

如果使用此代码，请引用：

```bibtex
@article{demeo2025drugreflector,
  title={Active learning framework leveraging transcriptomics identifies modulators of disease phenotypes},
  author={DeMeo, Benjamin and others},
  journal={Science},
  year={2025},
  doi={10.1126/science.adi8577}
}
```

---

## 联系与支持

如遇问题，请检查：
1. 数据预处理是否正确完成
2. GPU驱动和CUDA版本是否兼容
3. PyTorch版本 ≥ 1.12.0
4. 足够的磁盘空间（至少50GB）

**需要帮助？**
- 检查错误日志
- 确认数据格式
- 验证超参数设置